<a href="https://colab.research.google.com/github/raman-1106/2453-18-733-122_project1/blob/main/twitter_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reading a text-based dataset into pandas

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objects as go
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='latin-1')
df.head()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Remove the Unnamed: 2, Unnamed: 3, Unnamed: 4 columns due to all the entries were null.

In [ ]:
# see the null data here
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# search the most relevant message 
df['Message'].describe()

In [ ]:
# count of ham and spam
df['Category'].value_counts()

In [ ]:
# convert categorical Category to numerical with new column
df['Category_nm'] = df.Category.map({'ham':0, 'spam':1})
df.head()

In [ ]:
# interactive plotly hist plot for numerical vi_nm columns(i,e ham and spam)
df['Category_nm'].iplot(kind='hist')

In [ ]:
# creating a new column with message length using Message column
df['Message_le'] = df.Message.apply(len)
df.head()

In [ ]:
# Histogram plot for spam and ham labels with respeect to message length
plt.figure(figsize=(12,8))
df[df['Category']=='ham'].Message_le.plot(bins = 50, kind= 'hist', color='blue', label='ham', alpha=0.75)
df[df['Category']=='spam'].Message_le.plot(bins=50, kind= 'hist', color='red', label = 'spam', alpha=0.75)
plt.legend()
plt.xlabel('Message length')

In [ ]:
# describe the ham for some numerical insights
df[df['Category']=='ham'].describe()

In [ ]:
# describe the spam some numerical insights
df[df['Category']=='spam'].describe()

In [ ]:
# describe the both numerical columns
df.describe()

In [ ]:
# see in describe we have 910 word message, let's look at it
df[df['Message_le']==910].Message.iloc[0]

# Text Pre-processing

Our main issue with our data is that it is all in text format (strings). The classification algorithms that we usally use need some sort of numerical feature vector in order to perform the classification task. There are actually many methods to convert a corpus to a vector format. The simplest is the bag-of-words approach, where each unique word in a text will be represented by one number.

In this section we'll convert the raw messages (sequence of characters) into vectors (sequences of numbers).

As a first step, let's write a function that will split a message into its individual words and return a list. We'll also remove very common words, ('the', 'a', etc..). To do this we will take advantage of the NLTK library. It's pretty much the standard library in Python for processing text and has a lot of useful features. We'll only use some of the basic ones here.

Let's create a function that will process the string in the message column, then we can just use apply() in pandas do process all the text in the DataFrame.

First removing punctuation. We can just take advantage of Python's built-in string library to get a quick list of all the possible punctuation:

In [ ]:
import string 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

Now let's "tokenize" these messages. Tokenization is just the term used to describe the process of converting the normal text strings in to a list of tokens (words that we actually want).

In [ ]:
df['clean_msg'] = df.Message.apply(text_process)

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['clean_msgs'] = df.clean_msg.apply(lemmatizer.lemmatize)

In [ ]:
df.head()

In [ ]:
type(stopwords.words('english'))

In [ ]:
from collections import Counter

words = df[df['Category']=='ham'].clean_msg.apply(lambda x: [word.lower() for word in x.split()])
ham_words = Counter()

for msg in words:
    ham_words.update(msg)
    
print(ham_words.most_common(50))    

In [ ]:
words = df[df.Category=='spam'].clean_msg.apply(lambda x: [word.lower() for word in x.split()])
spam_words = Counter()

for msg in words:
    spam_words.update(msg)
    
print(spam_words.most_common(50))

In [ ]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = df.clean_msg
y = df.Category_nm
print(X.shape)
print(y.shape)

In [ ]:
# split X and y into training and testing sets 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

There are a lot of arguments and parameters that can be passed to the CountVectorizer. In this case we will just specify the analyzer to be our own previously defined function:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(X_train)

In [ ]:
# learn training data vocabulary, then use it to create a document-term matrix
X_train_dtm = vect.transform(X_train)



# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

# examine the document-term matrix
X_train_dtm

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)

# Building and evaluating a model

We will use multinomial Naive Bayes:

*    The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.


In [ ]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

In [ ]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

In [ ]:
# print message text for false positives (ham incorrectly classifier)
# X_test[(y_pred_class==1) & (y_test==0)]
X_test[y_pred_class > y_test]

In [ ]:
# print message text for false negatives (spam incorrectly classifier)
X_test[y_pred_class < y_test]

In [ ]:
# example of false negative 
X_test[4949]

In [ ]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)*100

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test, y_pred)*100

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

# Tuning the vectorizer

Thus far, we have been using the default parameters of CountVectorizer:

In [ ]:
# show default parameters for CountVectorizer
vect

However, the vectorizer is worth tuning, just like a model is worth tuning! Here are a few parameters that you might want to tune:

*    stop_words: string {'english'}, list, or None (default)
*        If 'english', a built-in stop word list for English is used.
*        If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens.
*        If None, no stop words will be used.

In [ ]:
# remove English stop words
vect = CountVectorizer(stop_words='english')

*    ngram_range: tuple (min_n, max_n), default=(1, 1)
*        The lower and upper boundary of the range of n-values for different n-grams to be extracted.
*        All values of n such that min_n <= n <= max_n will be used.

In [ ]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))

*    max_df: float in range [0.0, 1.0] or int, default=1.0
*        When building the vocabulary, ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words).
*        If float, the parameter represents a proportion of documents.
*        If integer, the parameter represents an absolute count.

In [ ]:
# ignore terms that appear in more than 50% of the documents
vect = CountVectorizer(max_df=0.5)

*    min_df: float in range [0.0, 1.0] or int, default=1
*        When building the vocabulary, ignore terms that have a document frequency strictly lower than the given threshold. (This value is also called "cut-off" in the literature.)
*        If float, the parameter represents a proportion of documents.
*        If integer, the parameter represents an absolute count.

In [ ]:
# only keep terms that appear in at least 2 documents
vect = CountVectorizer(min_df=2)

*    Guidelines for tuning CountVectorizer:
*        Use your knowledge of the problem and the text, and your understanding of the tuning parameters, to help you decide what parameters to tune and how to tune them.
*        Experiment, and let the data tell you the best approach!